In [59]:
#Ranking simple
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://tennisabstract.com/reports/atpRankings.html"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Encontrar la tabla del ranking
tabla = soup.find("table", {"id": "reportable"})
filas = tabla.find_all("tr")[1:]  # Saltamos encabezado

datos = []
for fila in filas:
    celdas = fila.find_all("td")
    if len(celdas) < 4:
        continue
    ranking = celdas[0].get_text(strip=True)
    jugador_link = celdas[1].find("a")
    jugador = jugador_link.get_text(strip=True).replace('\xa0', ' ')  # maneja &nbsp;
    url_jugador = jugador_link['href']
    pais = celdas[2].get_text(strip=True)
    nacimiento = celdas[3].get_text(strip=True)

    datos.append({
        "Ranking": int(ranking),
        "Jugador": jugador,
        "País": pais,
    })

Ranking_Actualizado = pd.DataFrame(datos)

def determinar_cotizacion(ranking):
    if ranking <= 4:
        return 60000
    elif ranking <= 10:
        return 30000
    elif ranking <= 20:
        return 20000
    elif ranking <= 30:
        return 12000
    elif ranking <= 50:
        return 5000
    elif ranking <= 100:
        return 3000
    else:
        return 1000

Ranking_Actualizado["Cotización"] = Ranking_Actualizado["Ranking"].apply(determinar_cotizacion)

Ranking_Actualizado.head(200)

,Ranking,Jugador,País,Cotización
0,1,Jannik Sinner,ITA,60000
1,2,Carlos Alcaraz,ESP,60000
2,3,Alexander Zverev,GER,60000
3,4,Taylor Fritz,USA,60000
4,5,Jack Draper,GBR,30000
...,...,...,...,...
195,196,Luca Van Assche,FRA,1000
196,197,Jason Kubler,AUS,1000
197,198,Yu Hsiou Hsu,TPE,1000
198,199,Jay Clarke,GBR,1000


In [119]:
#Ranking con último resultado
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. Obtener el ranking de jugadores
url_ranking = "https://tennisabstract.com/reports/atpRankings.html"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url_ranking, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

tabla = soup.find("table", {"id": "reportable"})
filas = tabla.find_all("tr")[1:]

ranking_data = []
for fila in filas:
    celdas = fila.find_all("td")
    if len(celdas) < 4:
        continue
    jugador_link = celdas[1].find("a")
    jugador_nombre = jugador_link.get_text(strip=True).replace('\xa0', ' ')
    jugador_id = jugador_nombre.replace(" ", "")  # 👈 Eliminar espacios

    ranking_data.append({
        "Jugador": jugador_nombre,
        "ID": jugador_id
    })

ranking_df = pd.DataFrame(ranking_data)

# 2. Función para dividir el ID en posibles apellidos
def generar_posibles_apellidos(nombre_compuesto):
    mayus_indices = [i for i, c in enumerate(nombre_compuesto) if c.isupper()]
    palabras = []
    for i in range(len(mayus_indices)):
        inicio = mayus_indices[i]
        fin = mayus_indices[i+1] if i+1 < len(mayus_indices) else len(nombre_compuesto)
        palabra = nombre_compuesto[inicio:fin]
        palabras.append(palabra)
    posibles = []
    for n in range(1, 4):
        if len(palabras) >= n:
            apellido = ' '.join(palabras[-n:])
            posibles.append(apellido)
    return posibles

# 3. Función que obtiene el último resultado para un jugador
def obtener_ultimo_resultado(jugador_id):
    url = f"https://www.tennisabstract.com/jsfrags/{jugador_id}.js"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.find_all("tr")

    posibles_apellidos = generar_posibles_apellidos(jugador_id)
    posibles_apellidos_lower = [ap.lower() for ap in posibles_apellidos]

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 8:
            continue

        torneo = cols[1].get_text(strip=True)
        ronda = cols[3].get_text(strip=True)

        cell_html = str(cols[6])
        soup_cell = BeautifulSoup(cell_html, "html.parser")
        texto = " ".join(soup_cell.stripped_strings).lower()

        marcador = cols[7].get_text(strip=True)

        resultado = "Desconocido"
        rival = None

        apellido_encontrado = next((ap for ap in posibles_apellidos_lower if ap in texto), None)

        if apellido_encontrado and " d. " in texto:
            jugador_1, jugador_2 = map(str.strip, texto.split(" d. "))
            if apellido_encontrado in jugador_1:
                resultado = "Ganó"
                rival = jugador_2
            elif apellido_encontrado in jugador_2:
                resultado = "Perdió"
                rival = jugador_1

        data.append({
            "Torneo": torneo,
            "Ronda": ronda,
            "Rival": rival.title() if rival else None,
            "Resultado": resultado,
            "Marcador": marcador
        })

    if not data:
        return "Sin datos"

    ronda_reciente = data[0]["Ronda"]
    for fila in data:
        if fila["Resultado"] in ["Ganó", "Perdió"]:
            if fila["Resultado"] == "Ganó":
                return f"Ganó vs {fila['Rival']} en {fila['Torneo']} y avanza a {ronda_reciente}"
            else:
                return f"Perdió vs {fila['Rival']} en {ronda_reciente} de {fila['Torneo']}"
    return "Sin resultado claro"

# 4. Aplicar a los primeros N jugadores
top_n = 200  # Cambia si querés más jugadores
ranking_df = ranking_df.head(top_n)
ranking_df["Último resultado"] = ranking_df["ID"].apply(obtener_ultimo_resultado)

# 5. Mostrar resultados
print(ranking_df[["Jugador", "Último resultado"]])

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [8]:
Resultado_Torneo[Resultado_Torneo['Jugador']=='Juan Manuel Cerundolo']

,Jugador,Torneo,Ronda,Qualy
80,Juan Manuel Cerundolo,Gstaad,F,—


In [44]:
import pandas as pd

# Ruta al archivo Excel
ruta_excel = r"C:\Users\bruno\OneDrive\Documentos\Proyecto Fantasy\Tablas Fantasy.xlsx"

# Leer las dos hojas
torneos_df = pd.read_excel(ruta_excel, sheet_name="Torneos 2025")
puntajes_df = pd.read_excel(ruta_excel, sheet_name="Puntaje por Ronda")
torneos_ch_df = pd.read_excel(ruta_excel, sheet_name="Torneos CH 2025")

# Mostrar una vista previa (opcional)
print("Torneos 2025:")
print(torneos_df.head())

print("\nPuntaje por Ronda:")
print(puntajes_df.head())

print("\nTorneos CH 2025:")
print(torneos_ch_df.head())


Torneos 2025:
  Fecha Inicio  Fecha Fin           Torneo      Categoría Superficie  Fecha  \
0   2024-12-29 2025-01-05         Brisbane  ATP 250 corto    Cemento      1   
1   2024-12-29 2025-01-05        Hong Kong  ATP 250 corto    Cemento      1   
2   2025-01-06 2025-01-11         Adelaide  ATP 250 corto    Cemento      2   
3   2025-01-06 2025-01-11         Auckland  ATP 250 corto    Cemento      2   
4   2025-01-12 2025-01-26  Australian Open     Grand Slam    Cemento      3   

        Entry-List  
0         brisbane  
1        hong-kong  
2         adelaide  
3         auckland  
4  australian-open  

Puntaje por Ronda:
  Categoría  Campeon   F  SF  QF   R16  R32  R64  R128  Q  Q3  Q2  Q1
0     CH 50       50  25  14   8   4.0    0  NaN   NaN  3 NaN   1   0
1     CH 75       75  44  22  12   6.0    0  NaN   NaN  4 NaN   2   0
2    CH 100      100  50  25  14   7.0    0  NaN   NaN  4 NaN   2   0
3    CH 125      125  64  35  16   8.0    0  NaN   NaN  5 NaN   3   0
4    CH 175    

In [45]:
import pandas as pd
from datetime import datetime

def obtener_fecha_actual(torneos_df, torneos_ch_df):
    hoy = pd.to_datetime(datetime.today().date())

    # Unir los torneos ATP y Challenger
    torneos_df["Fecha Inicio"] = pd.to_datetime(torneos_df["Fecha Inicio"])
    torneos_df["Fecha Fin"] = pd.to_datetime(torneos_df["Fecha Fin"])
    torneos_ch_df["Fecha Inicio"] = pd.to_datetime(torneos_ch_df["Fecha Inicio"])
    torneos_ch_df["Fecha Fin"] = pd.to_datetime(torneos_ch_df["Fecha Fin"])

    torneos_completo = pd.concat([torneos_df, torneos_ch_df], ignore_index=True)

    # Filtrar por torneos activos hoy
    torneos_en_juego = torneos_completo[
        (torneos_completo["Fecha Inicio"] <= hoy) & (torneos_completo["Fecha Fin"] >= hoy)
    ]

    if torneos_en_juego.empty:
        return None, pd.DataFrame()
    
    fecha_actual = torneos_en_juego["Fecha"].iloc[0]
    return fecha_actual, torneos_en_juego


In [46]:
import pandas as pd
from datetime import datetime

def obtener_fecha_anterior(torneos_df, torneos_ch_df):
    hoy = pd.to_datetime(datetime.today().date())

    torneos_df["Fecha Inicio"] = pd.to_datetime(torneos_df["Fecha Inicio"])
    torneos_df["Fecha Fin"] = pd.to_datetime(torneos_df["Fecha Fin"])
    torneos_ch_df["Fecha Inicio"] = pd.to_datetime(torneos_ch_df["Fecha Inicio"])
    torneos_ch_df["Fecha Fin"] = pd.to_datetime(torneos_ch_df["Fecha Fin"])

    # Fechas válidas sólo se sacan de torneos_df como referencia
    fechas_anteriores = torneos_df[torneos_df["Fecha Fin"] < hoy]["Fecha"].unique()

    if len(fechas_anteriores) == 0:
        return None, pd.DataFrame()

    fecha_previa = max(fechas_anteriores)

    torneos_fecha_previa = pd.concat([
        torneos_df[torneos_df["Fecha"] == fecha_previa],
        torneos_ch_df[torneos_ch_df["Fecha"] == fecha_previa]
    ], ignore_index=True)

    return fecha_previa, torneos_fecha_previa


In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# 1. Función para generar posibles apellidos
def generar_posibles_apellidos(nombre_compuesto):
    mayus_indices = [i for i, c in enumerate(nombre_compuesto) if c.isupper()]
    palabras = []
    for i in range(len(mayus_indices)):
        inicio = mayus_indices[i]
        fin = mayus_indices[i+1] if i+1 < len(mayus_indices) else len(nombre_compuesto)
        palabra = nombre_compuesto[inicio:fin]
        palabras.append(palabra)
    posibles = []
    for n in range(1, 4):
        if len(palabras) >= n:
            apellido = ' '.join(palabras[-n:])
            posibles.append(apellido)
    return posibles

# 2. Obtener fecha previa y torneos válidos
def obtener_fecha_anterior(torneos_df, torneos_ch_df):
    hoy = pd.to_datetime(datetime.today().date())
    torneos_df["Fecha Inicio"] = pd.to_datetime(torneos_df["Fecha Inicio"])
    torneos_df["Fecha Fin"] = pd.to_datetime(torneos_df["Fecha Fin"])
    torneos_ch_df["Fecha Inicio"] = pd.to_datetime(torneos_ch_df["Fecha Inicio"])
    torneos_ch_df["Fecha Fin"] = pd.to_datetime(torneos_ch_df["Fecha Fin"])

    fechas_df = pd.concat([torneos_df, torneos_ch_df], ignore_index=True)
    fechas_df["Fecha"] = pd.to_datetime(fechas_df["Fecha"])
    fechas_anteriores = fechas_df[fechas_df["Fecha Fin"] < hoy]["Fecha"].unique()

    if len(fechas_anteriores) == 0:
        return None, pd.DataFrame()

    fecha_previa = max(fechas_anteriores)
    torneos_fecha_previa = fechas_df[fechas_df["Fecha"] == fecha_previa].copy()

    return fecha_previa, torneos_fecha_previa

# 3. Obtener torneos jugados por jugador en esa semana
def obtener_torneos_por_fecha(jugador_id, torneos_validos):
    url = f"https://www.tennisabstract.com/jsfrags/{jugador_id}.js"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    # 🔒 Limitar el HTML a la sección de "Recent Results" únicamente
    html_recortado = response.text.split('<h1 id="tour-years-h">')[0]
    soup = BeautifulSoup(html_recortado, "html.parser")
    rows = soup.find_all("tr")

    posibles_apellidos = generar_posibles_apellidos(jugador_id)
    posibles_apellidos_lower = [ap.lower() for ap in posibles_apellidos]

    data = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 8:
            continue

        torneo = cols[1].get_text(strip=True)
        ronda = cols[3].get_text(strip=True)
        cell_html = str(cols[6])
        soup_cell = BeautifulSoup(cell_html, "html.parser")
        texto = " ".join(soup_cell.stripped_strings).lower()
        marcador = cols[7].get_text(strip=True)

        resultado = "Desconocido"
        apellido_encontrado = next((ap for ap in posibles_apellidos_lower if ap in texto), None)

        if apellido_encontrado and " d. " in texto:
            jugador_1, jugador_2 = map(str.strip, texto.split(" d. "))
            if apellido_encontrado in jugador_1:
                resultado = "Ganó"
            elif apellido_encontrado in jugador_2:
                resultado = "Perdió"

        data.append({
            "Torneo": torneo,
            "Ronda": ronda,
            "Resultado": resultado
        })

    if not data:
        return []

    df = pd.DataFrame(data)
    df = df[df["Torneo"].isin(torneos_validos)]
    if df.empty:
        return []

    torneos_info = []
    for torneo in df["Torneo"].unique():
        partidos = df[df["Torneo"] == torneo]
        ronda_mas_reciente = partidos.iloc[0]["Ronda"]
        ultima_definida = next((p for _, p in partidos.iterrows() if p["Resultado"] in ["Ganó", "Perdió"]), None)

        if ultima_definida is not None:
            ronda_perdida = ultima_definida["Ronda"]
            if ultima_definida["Resultado"] == "Perdió":
                ronda_final = "—" if ronda_perdida in ["Q1", "Q2", "Q3"] else ronda_perdida
            elif ultima_definida["Resultado"] == "Ganó" and ronda_perdida == "F":
                ronda_final = "Campeón"
            else:
                ronda_final = ronda_mas_reciente
        else:
            ronda_final = f"{ronda_mas_reciente} (aún en carrera)"

        rondas_qualy = partidos[partidos["Ronda"].isin(["Q1", "Q2", "Q3"])]
        qualy_status = "—"
        if not rondas_qualy.empty:
            perdio_en = rondas_qualy[rondas_qualy["Resultado"] == "Perdió"]
            if not perdio_en.empty:
                qualy_status = perdio_en.iloc[0]["Ronda"]
            else:
                jugo_main_draw = any(partidos["Ronda"].str.startswith("R"))
                qualy_status = "Q" if jugo_main_draw else rondas_qualy.iloc[-1]["Ronda"]

        torneos_info.append({
            "Torneo": torneo,
            "Ronda": ronda_final,
            "Qualy": qualy_status
        })

    return torneos_info


# 4. Obtener el ranking de jugadores
url_ranking = "https://tennisabstract.com/reports/atpRankings.html"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url_ranking, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

tabla = soup.find("table", {"id": "reportable"})
filas = tabla.find_all("tr")[1:]

ranking_data = []
for fila in filas:
    celdas = fila.find_all("td")
    if len(celdas) < 4:
        continue
    jugador_link = celdas[1].find("a")
    jugador_nombre = jugador_link.get_text(strip=True).replace('\xa0', ' ')
    jugador_id = jugador_nombre.replace(" ", "")
    ranking_data.append({
        "Jugador": jugador_nombre,
        "ID": jugador_id
    })

ranking_df = pd.DataFrame(ranking_data)
ranking_df = ranking_df.head(200)  # Cambiá esto si querés más o menos jugadores

# 5. Simulá carga de torneos_df y torneos_ch_df (vos deberías cargar los reales)
# Acá va un ejemplo mínimo, reemplazalo con tus datos reales
# torneos_df = pd.read_csv("torneos.csv")
# torneos_ch_df = pd.read_csv("torneos_ch.csv")

# 🔁 Suponiendo que ya los tenés cargados en torneos_df y torneos_ch_df:
fecha_previa, torneos_fecha_previa = obtener_fecha_anterior(torneos_df, torneos_ch_df)
torneos_validos = set(torneos_fecha_previa["Torneo"])

# 6. Obtener resultados por jugador
resultados_finales = []

for _, row in ranking_df.iterrows():
    jugador = row["Jugador"]
    jugador_id = row["ID"]
    torneos_jugados = obtener_torneos_por_fecha(jugador_id, torneos_validos)

    if not torneos_jugados:
        resultados_finales.append({
            "Jugador": jugador,
            "Torneo": "No jugó",
            "Ronda": None,
            "Qualy": None
        })
    else:
        for torneo_info in torneos_jugados:
            resultados_finales.append({
                "Jugador": jugador,
                **torneo_info
            })

Resultado_Torneo = pd.DataFrame(resultados_finales)

# 7. Mostrar resultado
print(Resultado_Torneo)


              Jugador          Torneo Ronda Qualy
0       Jannik Sinner         No jugó  None  None
1      Carlos Alcaraz         No jugó  None  None
2    Alexander Zverev  Canada Masters    SF     —
3        Taylor Fritz  Canada Masters    SF     —
4         Jack Draper         No jugó  None  None
..                ...             ...   ...   ...
195   Luca Van Assche        Porto CH   R16     —
196      Jason Kubler         No jugó  None  None
197      Yu Hsiou Hsu    Lexington CH    SF     —
198        Jay Clarke         No jugó  None  None
199      Rei Sakamoto    Lexington CH   R16     —

[200 rows x 4 columns]


In [48]:
import pandas as pd

def calcular_puntajes(Resultado_Torneo, torneos_df, torneos_ch_df, puntajes_df):
    # Unir torneos ATP y Challenger
    torneos_completo = pd.concat([torneos_df, torneos_ch_df], ignore_index=True)

    # Merge para agregar la Categoría desde torneos_completo
    resultado_final = Resultado_Torneo.merge(
        torneos_completo[["Torneo", "Categoría"]],
        on="Torneo",
        how="left"
    )

    # Unificar nombres de columnas para facilitar búsqueda
    puntajes_df = puntajes_df.rename(columns={"Campeon": "Campeón"})

    # Función que calcula puntaje por fila
    def obtener_puntaje(row):
        if row["Torneo"] == "No jugó" or pd.isna(row["Categoría"]):
            return 0

        categoria = row["Categoría"]
        ronda = str(row["Ronda"]).strip()
        qualy = str(row["Qualy"]).strip()

        fila_puntaje = puntajes_df[puntajes_df["Categoría"] == categoria]
        if fila_puntaje.empty:
            return 0

        fila_puntaje = fila_puntaje.iloc[0]

        # Puntaje por ronda principal
        if ronda not in ["—", "-", "None", "nan", ""]:
            if ronda == "F" and row.get("Resultado", "").strip() == "Ganó":
                ronda_clave = "Campeón"
            else:
                ronda_clave = ronda

            puntaje_ronda = fila_puntaje.get(ronda_clave, 0)
            puntaje_ronda = 0 if pd.isna(puntaje_ronda) else puntaje_ronda
        else:
            puntaje_ronda = 0

        # Puntaje por Qualy
        puntaje_qualy = fila_puntaje.get(qualy, 0)
        puntaje_qualy = 0 if pd.isna(puntaje_qualy) else puntaje_qualy

        return puntaje_ronda + puntaje_qualy

    # Calcular puntaje por fila
    resultado_final["Puntaje"] = resultado_final.apply(obtener_puntaje, axis=1)

    return resultado_final


In [49]:
resultado_final = calcular_puntajes(Resultado_Torneo, torneos_df, torneos_ch_df, puntajes_df)

# Por ejemplo, para ver solo jugador y su puntaje:
Puntaje_Fecha_Jugador = resultado_final[["Jugador", "Torneo", "Ronda", "Qualy", "Puntaje"]]
print(Puntaje_Fecha_Jugador)

              Jugador          Torneo Ronda Qualy  Puntaje
0       Jannik Sinner         No jugó  None  None      0.0
1      Carlos Alcaraz         No jugó  None  None      0.0
2    Alexander Zverev  Canada Masters    SF     —    400.0
3        Taylor Fritz  Canada Masters    SF     —    400.0
4         Jack Draper         No jugó  None  None      0.0
..                ...             ...   ...   ...      ...
203   Luca Van Assche        Porto CH   R16     —      6.0
204      Jason Kubler         No jugó  None  None      0.0
205      Yu Hsiou Hsu    Lexington CH    SF     —     22.0
206        Jay Clarke         No jugó  None  None      0.0
207      Rei Sakamoto    Lexington CH   R16     —      6.0

[208 rows x 5 columns]


In [54]:
Puntaje_Fecha_Jugador[Puntaje_Fecha_Jugador['Jugador']=='Juan Pablo Ficovich']

,Jugador,Torneo,Ronda,Qualy,Puntaje
127,Juan Pablo Ficovich,Canada Masters,R64,Q,50.0


In [55]:
import pandas as pd

def formatear_torneo_ronda(row):
    torneo = row['Torneo']
    ronda = row['Ronda']
    qualy = row['Qualy']
    
    # Caso sin qualy
    if qualy in [None, 'None', '—', '']:
        if pd.isna(ronda) or ronda in [None, 'None', '—', '']:
            return f"{torneo}"
        else:
            return f"{torneo} ({ronda})"
    
    # Caso con qualy + ronda definida (ej: R32 + Q)
    if ronda not in [None, 'None', '—', '']:
        return f"{torneo} ({ronda} + Q)"
    
    # Caso solo qualy (Q1, Q2, Q3)
    return f"{torneo} ({qualy})"

def agregar_por_jugador(df):
    # Formatear la columna con torneo+ronda+qualy
    df['Torneo_Ronda'] = df.apply(formatear_torneo_ronda, axis=1)

    # Agrupar por jugador
    df_agg = df.groupby('Jugador').agg({
        'Torneo_Ronda': lambda x: ', '.join(x),
        'Puntaje': 'sum'
    }).reset_index()

    return df_agg

# Uso
Resultado_Fecha = agregar_por_jugador(Puntaje_Fecha_Jugador)
print(Resultado_Fecha)


                         Jugador              Torneo_Ronda  Puntaje
0                    Adam Walton      Canada Masters (R64)     30.0
1               Adrian Mannarino  Canada Masters (R64 + Q)     50.0
2    Alejandro Davidovich Fokina      Canada Masters (R16)    100.0
3               Alejandro Tabilo                   No jugó      0.0
4           Aleksandar Kovacevic     Canada Masters (R128)     10.0
..                           ...                       ...      ...
195                 Yu Hsiou Hsu         Lexington CH (SF)     22.0
196               Zachary Svajda    Lexington CH (Campeón)     75.0
197                Zhizhen Zhang                   No jugó      0.0
198                  Zizou Bergs     Canada Masters (R128)     10.0
199                Zsombor Piros                   No jugó      0.0

[200 rows x 3 columns]


In [16]:
Resultado_Fecha[Resultado_Fecha['Jugador']=='Juan Manuel Cerundolo']

,Jugador,Torneo_Ronda,Puntaje
109,Juan Manuel Cerundolo,Gstaad (F),165.0


In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

# —————————————————————————————————
# 1. Limpieza de nombres
def limpiar_nombre(jugador_raw):
    nombre = re.sub(r"^\([^)]*\)\s*", "", jugador_raw)  # quitar prefijos
    nombre = nombre.replace("-", " ")                 # guiones a espacios
    nombre = re.sub(r"'([A-Z])", lambda m: m.group(1).lower(), nombre)  # O'Connell -> Oconnell
    return nombre

# —————————————————————————————————
# 2. Fecha actual y torneos activos
def obtener_fecha_actual(torneos_df, torneos_ch_df):
    hoy = pd.to_datetime(datetime.today().date())
    for df in (torneos_df, torneos_ch_df):
        df["Fecha Inicio"] = pd.to_datetime(df["Fecha Inicio"])
        df["Fecha Fin"] = pd.to_datetime(df["Fecha Fin"])
    torneos_completo = pd.concat([torneos_df, torneos_ch_df], ignore_index=True)
    en_juego = torneos_completo[
        (torneos_completo["Fecha Inicio"] <= hoy) & (torneos_completo["Fecha Fin"] >= hoy)
    ]
    if en_juego.empty:
        return None, pd.DataFrame()
    return en_juego["Fecha"].iloc[0], en_juego

# —————————————————————————————————
# 3. Extraer jugadores de un torneo por slug
def extraer_entry_list(slug, torneo_nombre):
    url = f"https://www.dartsrankings.com/tennis/{slug}"
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(r.text, "html.parser")
    tablas = soup.find_all("table")
    resultados = []
    for idx, fase in enumerate(["Main", "Qualy"]):
        if idx >= len(tablas): break
        for fila in tablas[idx].find_all("tr")[1:]:
            celdas = fila.find_all("td")
            if len(celdas) >= 2:
                nombre = limpiar_nombre(celdas[1].get_text(strip=True))
                resultados.append({
                    "Slug": slug,
                    "Torneo": torneo_nombre,
                    "Jugador": nombre,
                    "Cuadro": fase
                })
    return resultados

# —————————————————————————————————
# 4. Ranking Top 200 (desde TennisAbstract)
def obtener_top200():
    url = "https://tennisabstract.com/reports/atpRankings.html"
    r = requests.get(url, headers={"User-Agent":"Mozilla/5.0"})
    soup = BeautifulSoup(r.content, "html.parser")
    filas = soup.find("table", id="reportable").find_all("tr")[1:201]
    datos = []
    for fila in filas:
        tds = fila.find_all("td")
        nombre = tds[1].get_text(strip=True).replace("\xa0"," ")
        datos.append({"Jugador": nombre, "Ranking": int(tds[0].get_text())})
    return pd.DataFrame(datos)

# —————————————————————————————————
# 5. Flujo principal
def flujo_final(torneos_df, torneos_ch_df, puntajes_df):
    fecha_actual, en_juego = obtener_fecha_actual(torneos_df, torneos_ch_df)
    if fecha_actual is None:
        print("No hay torneos hoy")
        return pd.DataFrame()

    # Slugs y nombres de torneo
    slugs = en_juego["Entry-List"].unique()
    slug_map = en_juego.set_index("Entry-List")["Torneo"].to_dict()

    # Extraer jugadores por torneo
    entrada = []
    for slug in slugs:
        torneo_nombre = slug_map.get(slug, slug)
        entrada += extraer_entry_list(slug, torneo_nombre)

    df_entrada = pd.DataFrame(entrada)

    # Ranking
    df_top = obtener_top200()

    # Merge
    df = df_top.merge(df_entrada, on="Jugador", how="left")

    df["Juega"] = df["Slug"].notna()
    df["Cuadro"].fillna("-", inplace=True)
    df["Torneo"] = df["Torneo"].fillna("No juega")

    return df[["Ranking", "Jugador", "Juega", "Cuadro", "Torneo"]]

# —————————————————————————————————
# 6. Uso – reemplazar con tus tablas reales
# torneos_df = pd.read_excel(...); torneos_ch_df = ...
# puntajes_df = pd.read_excel(...)

Juegan_Fecha_Actual = flujo_final(torneos_df, torneos_ch_df, puntajes_df)
display(Juegan_Fecha_Actual)


,Ranking,Jugador,Juega,Cuadro,Torneo
0,1,Jannik Sinner,True,Main,Cincinnati Masters
1,2,Carlos Alcaraz,True,Main,Cincinnati Masters
2,3,Alexander Zverev,True,Main,Cincinnati Masters
3,4,Taylor Fritz,True,Main,Cincinnati Masters
4,5,Jack Draper,False,-,No juega
...,...,...,...,...,...
199,196,Luca Van Assche,False,-,No juega
200,197,Jason Kubler,False,-,No juega
201,198,Yu Hsiou Hsu,True,Main,Chicago CH
202,199,Jay Clarke,True,Main,Cordenons CH


In [57]:
Juegan_Fecha_Actual[Juegan_Fecha_Actual['Jugador']=='Juan Manuel Cerundolo']

,Ranking,Jugador,Juega,Cuadro,Torneo
85,86,Juan Manuel Cerundolo,True,Qualy,Cincinnati Masters
